In [26]:
import numpy
import urllib
import scipy.optimize
import random
import math
import sys
from sklearn.decomposition import PCA
from collections import defaultdict

In [27]:
def parseData(fname):
  for l in urllib.urlopen(fname):
    yield eval(l)

print "Reading data..."
data = list(parseData("file:///users/YW/Dropbox/UCSD/FA2015/CSE255/HW/hw2/beer_50000.json"))
print "done"

Reading data...
done


In [28]:
X = [[x['review/overall'], x['review/taste'], x['review/aroma'], x['review/appearance'], x['review/palate']] for x in data]

In [29]:
meanVector = []
for i in xrange(len(X[0])):
    meanVector.append(numpy.mean(map(lambda x: x[i], X)))

In [30]:
XMeanCompress = [meanVector for x in X]

In [31]:
def reconsError(X, XTrans):
    reconsError = 0;
    for i in xrange(len(X)):
        for j in xrange(len(X[i])):
            reconsError = reconsError + (X[i][j]-XTrans[i][j])**2
    return reconsError

In [32]:
reconsError(X, XMeanCompress)

113183.43370488819

In [33]:
pca = PCA(n_components=3)
pca.fit(X)

PCA(copy=True, n_components=3, whiten=False)

In [34]:
pca.components_

array([[ 0.48201817,  0.51561105,  0.44450326,  0.32114146,  0.44842921],
       [-0.46600016, -0.26162178,  0.31604305,  0.78071848, -0.07066358],
       [ 0.18730152, -0.11729702, -0.78013615,  0.424744  ,  0.4026657 ]])

In [35]:
XTrans = pca.transform(X)

In [36]:
XTransBack = pca.inverse_transform(XTrans)

In [37]:
reconsError(X, XTransBack)/len(X)

0.23428831021990038

In [59]:
Xa = X - numpy.mean(X, axis=0)

In [60]:
XaTrans = numpy.dot(Xa, pca.components_.T)
XaBack = numpy.dot(XaTrans, pca.components_)

In [61]:
reconsError(Xa, XaBack)/len(Xa)

0.23428831021990068

In [40]:
XCluster = X[0:500]

In [41]:
XCluster = [[x] for x in XCluster]

In [47]:
XCluster

[[[1.5, 1.5, 2.0, 2.5, 1.5]],
 [[3.0, 3.0, 2.5, 3.0, 3.0]],
 [[3.0, 3.0, 2.5, 3.0, 3.0]],
 [[3.0, 3.0, 3.0, 3.5, 2.5]],
 [[4.0, 4.5, 4.5, 4.0, 4.0]],
 [[3.0, 3.5, 3.5, 3.5, 3.0]],
 [[3.5, 4.0, 3.5, 3.5, 4.0]],
 [[3.0, 3.5, 2.5, 3.5, 2.0]],
 [[4.0, 4.0, 3.0, 3.5, 3.5]],
 [[4.5, 4.0, 3.5, 5.0, 4.0]],
 [[5.0, 4.0, 5.0, 4.0, 4.0]],
 [[4.0, 4.0, 4.0, 4.0, 3.5]],
 [[4.0, 3.0, 4.5, 3.0, 2.5]],
 [[3.5, 4.0, 4.0, 3.0, 3.0]],
 [[3.0, 3.0, 3.0, 2.5, 3.0]],
 [[2.0, 2.0, 1.5, 2.5, 2.5]],
 [[4.0, 4.0, 3.0, 4.0, 4.0]],
 [[4.5, 4.5, 4.5, 3.0, 4.0]],
 [[4.0, 4.0, 4.0, 4.0, 3.0]],
 [[4.0, 4.0, 4.5, 4.0, 3.5]],
 [[4.5, 4.5, 5.0, 4.5, 4.0]],
 [[4.0, 4.5, 4.5, 4.0, 4.0]],
 [[4.5, 4.5, 4.5, 4.0, 4.0]],
 [[5.0, 5.0, 5.0, 3.5, 3.5]],
 [[4.5, 4.5, 4.5, 4.0, 4.0]],
 [[4.5, 4.0, 5.0, 4.0, 4.0]],
 [[4.5, 4.5, 4.5, 3.5, 4.0]],
 [[4.5, 4.0, 4.5, 4.0, 4.0]],
 [[5.0, 5.0, 4.5, 4.0, 4.0]],
 [[4.0, 4.0, 4.0, 4.5, 4.0]],
 [[4.5, 4.0, 4.5, 4.0, 4.0]],
 [[4.0, 4.0, 4.0, 4.0, 4.0]],
 [[4.0, 4.0, 4.0, 3.0, 4.0]],
 [[3.0, 3.

In [38]:
def cluster(X):
    
    dists = [[0] * len(X) for row in xrange(len(X))]
    for i in xrange(len(X)):
        for j in xrange(len(X)):
            if i != j:
                dists[i][j] = (X[i][len(X[i])-1]-X[j][len(X[j])-1]).dot(X[i][len(X[i])-1]-X[j][len(X[j])-1])
            else:
                dists[i][j] = sys.float_info.max
    clu1 = X[numpy.where(dists==numpy.min(dists))[0][0]]
    clu2 = X[numpy.where(dists==numpy.min(dists))[1][0]]
    clu = clu1[0:-1] + clu2[0:-1]
    clu.append(numpy.mean(clu, axis=0))
    X.remove(clu1)
    X.remove(clu2)
    X.append(clu)
    print "clu1:"
    print clu1
    print "clu2:"
    print clu2
    return X


In [42]:
for x in XCluster:
    x.append(numpy.mean(x, axis=0))

In [43]:
while len(XCluster) > 2:
    cluster(XCluster)
    print len(XCluster)

clu1:
[[3.0, 3.0, 2.5, 3.0, 3.0], array([ 3. ,  3. ,  2.5,  3. ,  3. ])]
clu2:
[[3.0, 3.0, 2.5, 3.0, 3.0], array([ 3. ,  3. ,  2.5,  3. ,  3. ])]
499
clu1:
[[4.0, 4.5, 4.5, 4.0, 4.0], array([ 4. ,  4.5,  4.5,  4. ,  4. ])]
clu2:
[[4.0, 4.5, 4.5, 4.0, 4.0], array([ 4. ,  4.5,  4.5,  4. ,  4. ])]
498
clu1:
[[3.0, 3.5, 3.5, 3.5, 3.0], array([ 3. ,  3.5,  3.5,  3.5,  3. ])]
clu2:
[[3.0, 3.5, 3.5, 3.5, 3.0], array([ 3. ,  3.5,  3.5,  3.5,  3. ])]
497
clu1:
[[3.5, 4.0, 3.5, 3.5, 4.0], array([ 3.5,  4. ,  3.5,  3.5,  4. ])]
clu2:
[[3.5, 4.0, 3.5, 3.5, 4.0], array([ 3.5,  4. ,  3.5,  3.5,  4. ])]
496
clu1:
[[4.0, 4.0, 3.0, 3.5, 3.5], array([ 4. ,  4. ,  3. ,  3.5,  3.5])]
clu2:
[[4.0, 4.0, 3.0, 3.5, 3.5], array([ 4. ,  4. ,  3. ,  3.5,  3.5])]
495
clu1:
[[4.0, 4.0, 4.0, 4.0, 3.5], array([ 4. ,  4. ,  4. ,  4. ,  3.5])]
clu2:
[[4.0, 4.0, 4.0, 4.0, 3.5], array([ 4. ,  4. ,  4. ,  4. ,  3.5])]
494
clu1:
[[4.0, 4.0, 3.0, 4.0, 4.0], array([ 4.,  4.,  3.,  4.,  4.])]
clu2:
[[4.0, 4.0, 3.0, 4.0, 4.0]

In [44]:
XOrigin = X[0:500]

In [50]:
print len(XCluster)
print len(XCluster[0])-1
print XCluster[0]
print len(XCluster[1])-1
print XCluster[1]

2
9
[[1.0, 2.0, 4.0, 4.0, 2.0], [1.0, 1.0, 1.5, 1.0, 1.0], [2.0, 2.0, 2.5, 4.0, 2.5], [2.5, 2.5, 2.0, 2.5, 1.0], [2.0, 2.5, 2.5, 2.0, 2.5], [2.0, 2.0, 1.5, 2.5, 2.5], [1.5, 1.5, 2.0, 2.5, 1.5], [1.5, 1.5, 2.0, 3.0, 2.0], [1.5, 1.5, 1.5, 3.0, 2.0], array([ 1.66666667,  1.83333333,  2.16666667,  2.72222222,  1.88888889])]
491
[[4.5, 4.0, 2.0, 2.0, 4.0], [5.0, 5.0, 5.0, 3.0, 5.0], [3.5, 3.5, 3.5, 3.5, 5.0], [4.0, 5.0, 4.5, 5.0, 4.0], [3.5, 4.0, 5.0, 3.5, 3.5], [4.5, 5.0, 4.0, 4.5, 5.0], [5.0, 5.0, 4.5, 4.5, 5.0], [5.0, 5.0, 4.5, 4.5, 5.0], [5.0, 5.0, 4.5, 4.5, 4.5], [5.0, 5.0, 4.0, 4.5, 4.5], [5.0, 4.5, 5.0, 4.5, 4.5], [5.0, 4.5, 5.0, 5.0, 5.0], [5.0, 5.0, 5.0, 5.0, 5.0], [4.5, 4.5, 4.5, 3.0, 4.0], [4.5, 4.5, 4.5, 3.0, 4.5], [4.5, 4.5, 5.0, 3.5, 4.5], [4.5, 4.5, 4.5, 3.5, 4.5], [4.5, 4.5, 4.5, 4.0, 4.5], [4.5, 4.5, 5.0, 4.0, 4.5], [5.0, 5.0, 4.5, 4.0, 4.0], [5.0, 5.0, 4.5, 3.5, 4.0], [4.5, 5.0, 4.5, 3.5, 4.0], [4.5, 5.0, 4.0, 3.5, 4.0], [5.0, 5.0, 5.0, 3.5, 3.5], [5.0, 4.5, 5.0, 4.0, 4.0]

In [46]:
XRed = []

In [47]:
print len(XCluster[0])

10


In [48]:
for x in XOrigin:
    if x in XCluster[0][0:-1]:
        XRed.append(XCluster[0][len(XCluster[0])-1])
    else:
        XRed.append(XCluster[1][len(XCluster[1])-1])

In [49]:
reconsError(XOrigin, XRed)/len(XOrigin)

1.2550355284000967